In [41]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# !aws s3api create-bucket --bucket football-data-kamil --create-bucket-configuration LocationConstraint=eu-north-1

In [43]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import sys
import logging
import ipytest
from pathlib import Path

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [44]:
import os
import logging

from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

# Update this variable to your bucket name. This name must be unique
# across all AWS accounts.
BUCKET = os.environ["BUCKET"]
S3_LOCATION = f"s3://{BUCKET}/football"

# To run this notebook in Local Model, this constant must be set to True.
# I'm trying to do this automatically by checking for a specific environment
# variable that is set by SageMaker when you run the notebook inside SageMaker
# Studio. 
# LOCAL_MODE = "SAGEMAKER_INTERNAL_IMAGE_URI" not in os.environ

LOCAL_MODE = False

# This variable will be used to determine the architecture of the
# local machine. If the machine is an ARM64 machine, you will need
# to build a custom Docker image using the setup notebook.
ARCHITECTURE = !(uname -m)

# This is a dummy role that will be ignored when we run the
# pipeline in Local Mode.
DUMMY_ROLE = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-11111111111111"

# We'll use these two variables to configure the steps that do not support
# Local Mode.
pipeline_session = PipelineSession() if not LOCAL_MODE else LocalPipelineSession(default_bucket=BUCKET)
execution_role = os.environ["ROLE"] if not LOCAL_MODE else DUMMY_ROLE

if LOCAL_MODE:
    config = {
        "session": pipeline_session,
        "instance_type": "local",
        "role": DUMMY_ROLE,

        # We need to use a custom Docker image when we run the pipeline
        # in Local Model on an ARM64 machine.
        "image": "sagemaker-tensorflow-training-toolkit-local" if ARCHITECTURE[0] == "arm64" else None,
        "framework_version": None if ARCHITECTURE[0] == "arm64" else "1.7-1",
        "py_version": None if ARCHITECTURE[0] == "arm64" else "py3",
    }
else:
    config = {
        "session": pipeline_session,
        "instance_type": "ml.c5.xlarge",
        "role": execution_role,
        "image": None,
        "framework_version": "1.7-1",
        "py_version": "py3",
    }

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

In [80]:
import boto3

s3_client = boto3.client('s3')
iam_client = boto3.client("iam")
sagemaker_client = boto3.client("sagemaker")

In [46]:
from sagemaker.s3 import S3Uploader

df_local_path = str(os.environ['DATA_FILEPATH_X'])
y_local_path = str(os.environ['DATA_FILEPATH_Y'])

# S3Uploader.upload(local_path=df_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=sagemaker_session)
# S3Uploader.upload(local_path=y_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=sagemaker_session)

In [47]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

In [48]:
# from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.retry import (
    StepRetryPolicy,
    StepExceptionTypeEnum,
    SageMakerJobExceptionTypeEnum,
    SageMakerJobStepRetryPolicy
)

retry_policy = {
    "ExceptionType": SageMakerJobExceptionTypeEnum.INTERNAL_ERROR,
    "IntervalSeconds": 2,
    "BackoffRate": 2,
    "MaxAttempts": 5,
    "ExpireAfterMin": 5
}

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

processing_repository_uri = os.environ["PREPROCESSING_IMAGE_URI"]

from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(
    command=["python3"],
    image_uri=processing_repository_uri,
    role=config['role'],
    instance_count=1,
    instance_type="ml.m5.xlarge",
)

split_and_transform_data_step = ProcessingStep(
    name="split-and-transform-data",
    processor=processor,
    code=f"{CODE_FOLDER}/preprocessor.py",
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
        ProcessingOutput(output_name="model", source="/opt/ml/processing/model"),

        # The baseline output points to the test set before transforming the data. This set
        # will be helpful to generate a quality baseline for the model performance.
        ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline"),
    ],
    cache_config=cache_config,
    retry_policies=[
        SageMakerJobStepRetryPolicy(
            exception_types=[SageMakerJobExceptionTypeEnum.INTERNAL_ERROR],
            interval_seconds=60,
            backoff_rate=2.0,
            max_attempts=5
        ),
    ]
)

In [49]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

session1_pipeline = Pipeline(
    name="session1-pipeline",
    parameters=[dataset_location],
    steps=[
        split_and_transform_data_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config['session'],
)

session1_pipeline.upsert(role_arn=config["role"])

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session1-pipeline',
 'ResponseMetadata': {'RequestId': '63048ff7-fe33-4987-84be-ce0169e7a975',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '63048ff7-fe33-4987-84be-ce0169e7a975',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Wed, 27 Mar 2024 14:14:06 GMT'},
  'RetryAttempts': 0}}

In [50]:
# session1_pipeline.start()

In [51]:
from sagemaker.xgboost import XGBoost

use_spot_instances = True
max_run = 2400
max_wait = 1200 if use_spot_instances else None
instance_type = "ml.m5.2xlarge"

estimator = XGBoost(
    base_job_name="training",
    entry_point=f"{CODE_FOLDER}/train.py",
    role=config['role'],
    instance_count=1,
    instance_type=config['instance_type'],
    framework_version=config['framework_version'],
    # image_uri=config["image"],
    disable_profiler=True,
    py_version=config['py_version'],
    use_spot_instances=use_spot_instances,
    max_run=1000,
    max_wait=max_wait,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [52]:
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput

train_model_step = TrainingStep(
    name="train-model",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

In [53]:
USE_TUNING_STEP = True and not LOCAL_MODE

In [54]:
from sagemaker.tuner import HyperparameterTuner, WarmStartConfig, WarmStartTypes
from sagemaker.parameter import IntegerParameter, ContinuousParameter

hyperparameter_ranges = {
    'eta': ContinuousParameter(min_value=0.05, max_value=0.3, scaling_type="Logarithmic"),
    'max_depth': IntegerParameter(min_value=5, max_value=15, scaling_type="Auto"),
    'subsample': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Auto"),
    'colsample_bytree': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Logarithmic"),
    'lambda': ContinuousParameter(min_value=5, max_value=12, scaling_type="Logarithmic"),
    'alpha': ContinuousParameter(min_value=1, max_value=10, scaling_type="Logarithmic"),
    'min_child_weight': ContinuousParameter(min_value=0.4, max_value=1.0, scaling_type="Auto"),
}

objective_type = "Maximize"
metric_definitions = [
    {'Name': 'validation:logloss',
     'Regex': '.*\[[0-9]+\].*#011validation_0-logloss:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'
     },
    {'Name': 'validation:f1',
     'Regex': 'F1 score: ([0-9\\.]+)'
     },
]
metric_name = "validation:f1"
strategy = "Bayesian"

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_jobs=8,
    max_parallel_jobs=2,
    early_stopping_type='Auto',
)

In [55]:
from sagemaker.workflow.steps import TuningStep

tune_model_step = TuningStep(
    name="tune-model",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)


In [56]:
session2_pipeline = Pipeline(
    name="session2-pipeline",
    parameters=[dataset_location],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session2_pipeline.upsert(role_arn=config['role'])

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session2-pipeline',
 'ResponseMetadata': {'RequestId': '2c12de41-a8a1-48be-bcd6-6abcd5499684',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2c12de41-a8a1-48be-bcd6-6abcd5499684',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Wed, 27 Mar 2024 14:14:10 GMT'},
  'RetryAttempts': 0}}

In [57]:
# session2_pipeline.start()

In [58]:
from sagemaker.xgboost import XGBoostProcessor

evaluation_processor = XGBoostProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=config['role'],
    sagemaker_session=config["session"],
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [59]:
model_assets = train_model_step.properties.ModelArtifacts.S3ModelArtifacts

if USE_TUNING_STEP:
    model_assets = tune_model_step.get_top_model_s3_uri(
        top_k=0, s3_bucket=config["session"].default_bucket()
    )

In [60]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

In [61]:
evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=evaluation_processor.run(
        inputs=[
            # The first input is the test split that we generated on
            # the first step of the pipeline when we split and
            # transformed the data.
            ProcessingInput(
                source=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            # The second input is the model that we generated on
            # the Training or Tunning Step.
            ProcessingInput(
                source=model_assets,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            # The output is the evaluation report that we generated
            # in the evaluation script.
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"{CODE_FOLDER}/evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

In [62]:
MODEL_PACKAGE_GROUP = os.environ.get("MODEL_PACKAGE_GROUP")

In [63]:
from sagemaker.xgboost.model import XGBoostModel

xgb_model = XGBoostModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=config["role"],
)

In [64]:
from sagemaker.model_metrics import ModelMetrics, MetricsSource
from sagemaker.workflow.functions import Join

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                evaluate_model_step.properties.ProcessingOutputConfig.Outputs[
                    "evaluation"
                ].S3Output.S3Uri,
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    )
)

In [65]:
from sagemaker.workflow.parameters import ParameterFloat
import json

response = sagemaker_client.list_model_packages(ModelPackageGroupName=MODEL_PACKAGE_GROUP,
                                                SortBy='CreationTime',
                                                SortOrder='Descending',
                                                MaxResults=1)

if response['ModelPackageSummaryList']:
    latest_model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
    model_package_description = sagemaker_client.describe_model_package(ModelPackageName=latest_model_package_arn)
    evaluation_metrics = model_package_description['ModelMetrics']['ModelQuality']['Statistics']

    s3_uri = evaluation_metrics['S3Uri']
    s3_bucket, s3_key = s3_uri.replace("s3://", "").split("/", 1)

    obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
    evaluation_json = json.loads(obj['Body'].read())
    f1_metric = evaluation_json['metrics']['f1']['value']

    f1_threshold = ParameterFloat(name="f1_threshold", default_value=f1_metric)
    print(f"F1 score from the latest model's evaluation: {f1_metric}")
else:
    print("No models found in the specified model package group.")
    f1_metric = 0.67
    f1_threshold = ParameterFloat(name="f1_threshold", default_value=f1_metric)

F1 score from the latest model's evaluation: 0.6708633093525179


In [66]:
from sagemaker.workflow.fail_step import FailStep

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ",
        values=[
            "Execution failed because the model's f1 score was lower than",
            f1_threshold,
        ],
    ),
)

In [67]:
from sagemaker.workflow.model_step import ModelStep

register_model_step = ModelStep(
    name="register-model",
    step_args=xgb_model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        approval_status="Approved",
        model_metrics=model_metrics,
        content_types=["text/csv"],
        response_types=["application/json"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="XGBOOST",
        framework_version=config["framework_version"],
    ),
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [68]:
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo

condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluate_model_step.name,
        property_file=evaluation_report,
        json_path="metrics.f1.value",
    ),
    right=f1_threshold,
)

from sagemaker.workflow.condition_step import ConditionStep

condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step] if not LOCAL_MODE else [],
    else_steps=[fail_step],
)

In [69]:
session3_pipeline = Pipeline(
    name="session3-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        train_model_step,
        tune_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session3_pipeline.upsert(role_arn=config['role'])

INFO:sagemaker.processing:Uploaded None to s3://sagemaker-eu-north-1-284415450706/session3-pipeline/code/4e2d985798536d66f15bf6c82be000f9/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-north-1-284415450706/session3-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-eu-north-1-284415450706/session3-pipeline/code/4e2d985798536d66f15bf6c82be000f9/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-north-1-284415450706/session3-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session3-pipeline',
 'ResponseMetadata': {'RequestId': '636a0bd1-2c26-4b9f-a197-882dd90e369a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '636a0bd1-2c26-4b9f-a197-882dd90e369a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Wed, 27 Mar 2024 14:14:15 GMT'},
  'RetryAttempts': 0}}

In [70]:
# session3_pipeline.start()

In [71]:
from sagemaker.predictor import Predictor

ENDPOINT = "football-endpoint"
DATA_CAPTURE_DESTINATION = f"{S3_LOCATION}/monitoring/data-capture"

In [72]:
transformation_pipeline_model = Join(
    on="/",
    values=[
        split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "model"
        ].S3Output.S3Uri,
        "model.tar.gz",
    ],
)

In [73]:
from sagemaker.sklearn.model import SKLearnModel

preprocessing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="preprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)

In [74]:
post_processing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="postprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)

In [75]:
from sagemaker.pipeline import PipelineModel

pipeline_model = PipelineModel(
    name="inference-model",
    models=[preprocessing_model, xgb_model, post_processing_model],
    sagemaker_session=config["session"],
    role=config['role'],
)

In [76]:
PIPELINE_MODEL_PACKAGE_GROUP = "pipeline"

register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=pipeline_model.register(
        model_package_group_name=PIPELINE_MODEL_PACKAGE_GROUP,
        model_metrics=model_metrics,
        approval_status="PendingManualApproval",
        content_types=["text/csv", "application/json"],
        response_types=["text/csv", "application/json"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="XGBOOST",
        framework_version=config["framework_version"],
    ),
)

In [77]:
condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step] if not LOCAL_MODE else [],
    else_steps=[fail_step],
)

In [78]:
session4_pipeline = Pipeline(
    name="session4-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session4_pipeline.upsert(role_arn=config['role'])

INFO:sagemaker.processing:Uploaded None to s3://sagemaker-eu-north-1-284415450706/session4-pipeline/code/4e2d985798536d66f15bf6c82be000f9/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-north-1-284415450706/session4-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session4-pipeline',
 'ResponseMetadata': {'RequestId': 'ca1aa737-1b3d-4191-b6b3-ab1c8914dbad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ca1aa737-1b3d-4191-b6b3-ab1c8914dbad',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Wed, 27 Mar 2024 14:14:17 GMT'},
  'RetryAttempts': 0}}

In [79]:
session4_pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session4-pipeline/execution/yis36zi92476', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x00000168EEEE51F0>)

In [ ]:
lambda_role_name = "lambda-deployment-role"
lambda_role_arn = None

try:
    response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(
            {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": ["lambda.amazonaws.com", "events.amazonaws.com"]
                        },
                        "Action": "sts:AssumeRole",
                    }
                ],
            }
        ),
        Description="Lambda Endpoint Deployment",
    )

    lambda_role_arn = response["Role"]["Arn"]

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
        RoleName=lambda_role_name,
    )

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
        RoleName=lambda_role_name,
    )

    print(f'Role "{lambda_role_name}" created with ARN "{lambda_role_arn}".')
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName=lambda_role_name)
    lambda_role_arn = response["Role"]["Arn"]
    print(f'Role "{lambda_role_name}" already exists with ARN "{lambda_role_arn}".')

In [68]:
# from sagemaker import ModelPackage
# 
# response = sagemaker_client.list_model_packages(
#     ModelPackageGroupName=MODEL_PACKAGE_GROUP,
#     ModelApprovalStatus="Approved",
#     SortBy="CreationTime",
#     MaxResults=1,
# )
# 
# package = (
#     response["ModelPackageSummaryList"][0]
#     if response["ModelPackageSummaryList"]else None
# )
# 
# if package:
#     model_package = ModelPackage(
#         model_package_arn=package["ModelPackageArn"],
#         sagemaker_session=config['session'],
#         role=config['role'],
#     )
#     
#     
# package

{'ModelPackageGroupName': 'football',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:eu-north-1:284415450706:model-package/football/2',
 'CreationTime': datetime.datetime(2024, 3, 25, 18, 35, 48, 561000, tzinfo=tzlocal()),
 'ModelPackageStatus': 'Completed',
 'ModelApprovalStatus': 'Approved'}